In [1]:
import math
from interval import interval, imath, fpu

In [2]:
class Taylor:
    def __init__(self, coef:list, order:int):
        self.order=order
        if len(coef)<(order+1):
            for i in range(order+1-len(coef)):
                coef.append(0)
        #self.coef=[float(i) for i in coef]
        self.coef=coef

    def __repr__(self):
        return 'Taylor(%s, %d)'%(self.coef, self.order)
   
    def fixshape(x, y):
        order=max(x.order,y.order)
        x1=x.coef[:]
        y1=y.coef[:]
        if x.order<order:
            for i in range(order-x.order):
                x1.append(0)
        else:
            for i in range(order-y.order):
                y1.append(0)
        return Taylor(x1,order), Taylor(y1,order), order
        
    def fixshape1(x, y):
        order=x.order+y.order
        x1=x.coef[:]
        y1=y.coef[:]
        if x.order<order:
            for i in range(order-x.order):
                x1.append(0)
        else:
            for i in range(order-y.order):
                y1.append(0)
        return Taylor(x1,order), Taylor(y1,order), order   
    
    def firstnonzero(self):
        order=self.order
        nonzero=order+1
        for i in range(order+1):
            if self.coef[i]!=0:
                nonzero=i
                break
        return nonzero
    
    def __add__(self, other):
        coef=[]
        self1, other1, order=self.fixshape(other)
        for i in range(order+1):
            coef.append(self1.coef[i]+other1.coef[i])
        return Taylor(coef,order)
    
    def __sub__(self, other):
        coef=[]
        self1, other1, order=self.fixshape(other)
        for i in range(order+1):
            coef.append(self1.coef[i]-other1.coef[i])
        return Taylor(coef,order)
    
    def __mul__(self, other):
        coef=[]
        self1, other1, order=Taylor.fixshape1(self, other)
        coef.append(self1.coef[0]*other1.coef[0])
        for i in range(1,order+1):
            s=0
            for j in range(i+1):
                s+=self1.coef[j]*other1.coef[i-j]
            coef.append(s)
        return Taylor(coef,order)
    
    def __truediv__(self, other):
        self1, other1, order=self.fixshape(other)
        coef=Taylor.zero(order).coef
        orddivfact, cdivfact=self1.divfactorization(other1)
        coef[0]=cdivfact
        for i in range(orddivfact+1,order+1):
            coef[i-orddivfact]=self.divcoef(i,self1.coef,other1.coef,coef,orddivfact)
        return Taylor(coef,order)
    
    def divfactorization(self,other):
        a1nz=self.firstnonzero()
        b1nz=other.firstnonzero()
        orddivfact=min(a1nz,b1nz)
        try:
            cdivfact=self.coef[orddivfact]/other.coef[orddivfact]
            aux=cdivfact**2
        except ZeroDivisionError:
            print("Division does not define a Taylor polynomial\n")
        return orddivfact, cdivfact
    
    def divcoef(self,i,coef1,coef2,coef,orddivfact):
        s=0
        for j in range(i+1):
            s+=coef[j]*coef2[i-j]
        s=(coef1[i]-s)/coef2[orddivfact]
        return s

    def zero(n):
        coef=[]
        coef.append(0)
        for i in range(n):
            coef.append(0)
        return Taylor(coef,n)
    
    def one(n):
        coef=[]
        coef.append(1)
        for i in range(n):
            coef.append(0)
        return Taylor(coef,n)
    
    def __eq__(self,other):
        self1, other1, order=Taylor.fixshape(self, other)
        return self1.coef==other1.coef
    
    def square(self):
        order=self.order*2
        for a in range(self.order):
            self.coef.append(0)
        coef=[]
        coef.append((self.coef[0])**2)
        for i in range(1,order+1):
            coef.append(self.squarecoef(i,self.coef))
        return Taylor(coef,order)        
    
    def squarecoef(self,i,coef):
        s=0
        iend=int((i-2+i%2)/2)
        for j in range(iend+1):
            s+=coef[j]*coef[i-j]
        s=2*s
        if i%2==0:  
            s+=coef[int(i/2)]**2
        return s
    
    def sqrt(self):
        order=self.order
        l0nz=self.firstnonzero()
        if l0nz%2==1:
            raise ValueError("First non-vanishing Taylor coefficient must be an EVEN POWER to expand SQRT around 0.\n")
        lnull=int(l0nz/2)
        aux=math.sqrt(self.coef[l0nz])
        coef=Taylor.zero(order).coef
        coef[lnull]=aux
        for i in range(lnull+1,order-l0nz+1):
            coef[i]=self.sqrtcoef(i,self.coef,coef,lnull)
        return Taylor(coef,order)

    def sqrtcoef(self,i,ac,coef,lnull):
        s=0
        kodd=(i-lnull)%2
        kend=int((i-lnull-2+kodd)/2)
        for j in range(lnull+1,lnull+kend+1):
            s+=coef[j]*coef[i+lnull-j]
        aux=ac[i+lnull]-2*s
        if kodd==0:
            aux=aux-(coef[kend+lnull+1])**2
        s=aux/(2*coef[lnull])
        return s
        
    
    def __pow__(self,n:int):
        if type(n)==int:
            order=self.order*n
            for a in range(self.order):
                self.coef.append(0)
            if n<0:
                return Taylor.one(n)/self.__pow__(-n)
            elif n==0:
                return Taylor.one(order)
            elif n%2==0:
                if n==2:
                    return self.square()
                else:
                    p=int(n/2)
                    return Taylor(self.__pow__(p).square().coef,order)
            elif n==1:
                return self
            else:
                p=int((n-1)/2)
                return Taylor((self*self.__pow__(p).square()).coef,order)
        elif type(n)==float:
            order=self.order
            uno=Taylor.one(order)
            if n==0:
                return uno
            elif n==0.5:
                return self.sqrt()
            l0nz=self.firstnonzero()
            lnull=n*l0nz
            lnull=int(lnull)
            aux=(self.coef[l0nz])**n
            coef=Taylor.zero(order).coef
            coef[lnull]=aux
            k0=lnull+l0nz
            for i in range(k0+1,order+1):
                coef[i-l0nz]=self.powcoef(i,self.coef,n,coef,l0nz)
            return Taylor(coef,order)

    def powcoef(self,i,ac,x,coef,l0nz):
        s=0
        for j in range(i-l0nz):
            aux=x*(i-j)-j
            s+=aux*ac[i-j]*coef[j]
        aux=i-l0nz*(x+1)
        s=s/(aux*ac[l0nz])
        return s
    
    def exp(self):
        order=self.order
        coef=[]
        coef.append(math.exp(self.coef[0]))
        for i in range(1,order+1):
            s=0
            for j in range(i):
                s+=(i-j)*self.coef[i-j]*coef[j]
            coef.append(s/i)
        return Taylor(coef,order)
    
    def log(self):
        order=self.order
        coef=[]
        coef.append(math.log(self.coef[0]))
        for i in range(1,order+1):
            s=0
            for j in range(1,i):
                s+=(i-j)*self.coef[j]*coef[i-j]
            coef.append((self.coef[i]-s/i)/self.coef[0])
        return Taylor(coef,order)
    
    def sincos(self):
        order=self.order
        scoef=[];ccoef=[]
        scoef.append(math.sin(self.coef[0]))
        ccoef.append(math.cos(self.coef[0]))
        for i in range(1,order+1):
            s=0;c=0
            for j in range(1,i+1):
                s+=j*self.coef[j]*ccoef[i-j]
                c-=j*self.coef[j]*scoef[i-j]
            scoef.append(s/i)
            ccoef.append(c/i)
        return Taylor(scoef,order), Taylor(ccoef,order)
    
    def sin(self):
        return self.sincos()[0]
    
    def cos(self):
        return self.sincos()[1]
    
    def tan(self):
        order=self.order
        coef1=[]
        coef2=[]
        t=math.tan(self.coef[0])
        coef1.append(t)
        coef2.append(t**2)
        for i in range(1,order+1):
            coef1.append(self.tancoef(i,self.coef,coef2))
            coef2.append(self.squarecoef(i,coef1))
        return Taylor(coef1,order)
    
    def tancoef(self,i,coef,coef2):
        s=0
        for j in range(i):
            s+=(i-j)*coef[i-j]*coef2[j]
        s=self.coef[i]+s/i
        return s
    
    def diffTaylor(self):
        order=self.order
        coef=[]
        coef.append(self.coef[1])
        for i in range(2,order+1):
            coef.append(i*self.coef[i])
        coef.append(0)
        return Taylor(coef,order)
        
    def inteTaylor(self,x):
        order=self.order
        coef=[]
        coef.append(x)
        for i in range(1,order+1):
            coef.append(self.coef[i-1]/i)
        return Taylor(coef,order)
     
    def __call__(self,dx):
        order=self.order
        s=self.coef[-1]
        for i in range(order-1,-1,-1):
            s=s*dx+self.coef[i]
        return s
        
    def deriv(self,n):
        s=math.factorial(n)*self.coef[n]
        return s

In [3]:
c=Taylor([0,1,2,3,4],4)
b=Taylor([1.0,2.0,3.0],2)
a=Taylor([0,1,0,0,0],4)

In [4]:
d=Taylor([0,1],17)
f=Taylor([1,1],5)
f

Taylor([1, 1, 0, 0, 0, 0], 5)

In [5]:
c+b+c

Taylor([1.0, 4.0, 7.0, 6, 8], 4)

In [6]:
c-b

Taylor([-1.0, -1.0, -1.0, 3, 4], 4)

In [7]:
c*b

Taylor([0.0, 1.0, 4.0, 10.0, 16.0, 17.0, 12.0], 6)

In [8]:
c/a

Taylor([1.0, 2.0, 3.0, 4.0, 0], 4)

In [9]:
Taylor([1],6)/(Taylor([1],5)-a)

Taylor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 6)

In [10]:
(Taylor([1],4)-a)

Taylor([1, -1, 0, 0, 0], 4)

In [11]:
Taylor.one(4)

Taylor([1, 0, 0, 0, 0], 4)

In [12]:
Taylor.zero(4)

Taylor([0, 0, 0, 0, 0], 4)

In [13]:
a.exp()

Taylor([1.0, 1.0, 0.5, 0.16666666666666666, 0.041666666666666664], 4)

In [14]:
(Taylor.one(1)-a).log()

Taylor([0.0, -1.0, -0.5, -0.3333333333333333, -0.25], 4)

In [15]:
a.sincos()

(Taylor([0.0, 1.0, 0.0, -0.16666666666666666, 0.0], 4),
 Taylor([1.0, 0.0, -0.5, 0.0, 0.041666666666666664], 4))

In [16]:
a.sin()

Taylor([0.0, 1.0, 0.0, -0.16666666666666666, 0.0], 4)

In [17]:
b.sin()

Taylor([0.8414709848078965, 1.0806046117362795, -0.062035052011373715], 2)

In [18]:
a.cos()

Taylor([1.0, 0.0, -0.5, 0.0, 0.041666666666666664], 4)

In [19]:
c.square()

Taylor([0, 0, 1, 4, 10, 20, 25, 24, 16], 8)

In [20]:
f.sqrt()

Taylor([1.0, 0.5, -0.125, 0.0625, -0.0390625, 0.02734375], 5)

In [21]:
b.exp()

Taylor([2.718281828459045, 5.43656365691809, 13.591409142295227], 2)

In [22]:
a.tan()

Taylor([0.0, 1.0, 0.0, 0.3333333333333333, 0.0], 4)

In [23]:
b.diffTaylor()

Taylor([2.0, 6.0, 0], 2)

In [24]:
a.exp().diffTaylor()

Taylor([1.0, 1.0, 0.5, 0.16666666666666666, 0], 4)

In [25]:
a.exp()

Taylor([1.0, 1.0, 0.5, 0.16666666666666666, 0.041666666666666664], 4)

In [26]:
b**0

Taylor([1], 0)

In [27]:
b**-1

Taylor([1.0, -2.0, 1.0], 2)

In [28]:
b**4==b*b*b*b

True

In [29]:
b**4

Taylor([1.0, 8.0, 36.0, 104.0, 214.0, 312.0, 324.0, 216.0, 81.0], 8)

In [30]:
a1=(Taylor([1],5)-a)
a1

Taylor([1, -1, 0, 0, 0, 0], 5)

In [31]:
a1**(3.2)

Taylor([1.0, -3.2, 3.5200000000000005, -1.4080000000000004, 0.07040000000000009, 0.011264000000000012], 5)

In [32]:
a**2

Taylor([0, 0, 1, 0, 0, 0, 0, 0, 0], 8)

In [33]:
b.inteTaylor(1)

Taylor([1, 1.0, 1.0], 2)

In [34]:
a.exp().inteTaylor(0)

Taylor([0, 1.0, 0.5, 0.16666666666666666, 0.041666666666666664], 4)

In [35]:
a.exp().diffTaylor().inteTaylor(1)==a.exp()

True

In [36]:
a.exp()(1)

2.708333333333333

In [37]:
a.exp()(2)

7.0

In [38]:
a.exp()(interval[1,2])

interval([2.708333333333333, 7.0])

In [39]:
a.exp()(1)-math.exp(1)

-0.009948495125712054

In [40]:
d.exp()(interval(1))

interval([2.7182818284590446, 2.718281828459045])

In [41]:
d.exp()(1)-math.exp(1)

0.0

In [42]:
f.exp().deriv(5)==math.exp(1)

True

In [43]:
f.exp()

Taylor([2.718281828459045, 2.718281828459045, 1.3591409142295225, 0.45304697140984085, 0.11326174285246021, 0.02265234857049204], 5)

In [44]:
(lambda x: x**2 + x + x**3 - x**2)(interval[-1,1])

interval([-3.0, 3.0])

In [45]:
f1=Taylor([0,1,1],2)
f2=Taylor([0,0,-1,1],3)

In [46]:
f3=f1+f2
f3(interval[-1,1])

interval([-2.0, 2.0])